## Setup

### Import libraries

In [1]:
from benchmarks.experiments.wrapper_overhead.common.deployment_utility import WrapperOverheadDeploymentUtility

### Setup Common Resources

In [5]:
region = 'us-east-2'
deployment_utility: WrapperOverheadDeploymentUtility = WrapperOverheadDeploymentUtility(region)

## Deploy Associated AWS functions

### DNA Visualization

#### Boto3 Direct Calls

In [ ]:
dna_vis_direct_calls_dir = "./dna_visualization/external_database/boto3_only_direct_calls" # Boto3 only direct calls
deployment_utility.deploy_experiment(dna_vis_direct_calls_dir)

#### Boto3 SNS calls

In [ ]:
dna_vis_sns_calls_dir = "./dna_visualization/external_database/boto3_only_sns" # Boto3 only sns calls
deployment_utility.deploy_experiment(dna_vis_sns_calls_dir)

#### AWS Step Function

In [ ]:
dna_vis_aws_step_functions_dir = "./dna_visualization/external_database/aws_step_function" # AWS Step Functions
deployment_utility.deploy_experiment(dna_vis_aws_step_functions_dir)

#### Multi-X

- For MultiX, please set it up according to readme contained in the file. "./dna_visualization/external_database/multi_x"